In [ ]:
import pandas as pd
from numba import cuda
from itertools import islice
import time
from tqdm import tqdm

In [ ]:
df = pd.read_csv('All_Cluster_Perspective.csv', dtype = 'str')

In [ ]:
from simpletransformers.classification import ClassificationModel
model_args= {
            "num_train_epochs": 15,
            "learning_rate": 1e-5,
            "max_seq_length": 512,
            "silent": True
            }

model = ClassificationModel(
    "xlmroberta", "classla/xlm-roberta-base-multilingual-text-genre-classifier", use_cuda=False,
    args=model_args
    
)



In [ ]:
predictions, logit_output = model.predict(['जय श्रीराम'])

In [ ]:
for i in predictions:
    gn = model.config.id2label[i]
print(gn)

In [ ]:
def predict(dataframe, dataframe_column, final_file):
    
    def chunk(arr_range, arr_size):
        arr_range = iter(arr_range)
        return iter(lambda: tuple(islice(arr_range, arr_size)), ())
    
    batches_list = list(chunk(dataframe[dataframe_column], 8))
    batches_list_new = []

    for i in batches_list:
        batches_list_new.append(list(i))

    print("The dataset is split into {} batches of {} texts.".format(len(batches_list_new),len(batches_list_new[0])))

    y_pred = []

    print("Prediction started.")
    start_time = time.time()

    for i in tqdm(batches_list_new):
        try:
            output = model.predict(i)
            current_y_pred = [model.config.id2label[i] for i in output[0]]

            for i in current_y_pred:
                y_pred.append(i)
        except:
            for i in range(0,8):
                y_pred.append(-1)

    prediction_time = round((time.time() - start_time)/60,2)

    print("\n\nPrediction completed. It took {} minutes for {} instances - {} minutes per one instance.".format(prediction_time, dataframe.shape[0], prediction_time/dataframe.shape[0]))

    dataframe["X-GENRE"] = y_pred

    dataframe.to_csv("{}".format(final_file))

    return dataframe


In [ ]:
start=120001
end=127484
em = 0

In [ ]:
test4 = predict(df.loc[start:end], 'Text_without_Link', 'Genre_Detection_4'+ '.csv')

In [ ]:
test4.to_csv('Genre_Detection_4.csv')

In [ ]:
test2

In [ ]:
df2 = pd.read_csv('Genre_Detection_0.csv')

In [ ]:
df2.groupby('X-GENRE').count()